# 链家
/Users/xusanshan/Downloads/com.homelink.android_9.17.0_9170100.apk


In [22]:
from appium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time, random, re
from appium.webdriver.common.touch_action import TouchAction
from appium.webdriver.common.mobileby import MobileBy
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from pymongo import MongoClient



# Appium 基本参数
PLATFORM = 'Android'
DEVICE_NAME = 'MI_PAD_4'
DRIVER_SERVER = 'http://localhost:4723/wd/hub'
TIMEOUT = 800
NORESET = 'True'
FLICK_STRAT_X = 300
FLICK_START_Y = 300
FLICK_DISTANCE = 700
MONGO_URL = 'localhost'
MONGO_DB = 'lianjia'
MONGO_COLLECTION = 'houses'

class Automation():
    # 初始化 Appium 基本参数
    def __init__(self, APP_PACKAGE, APP_ACTIVITY):
        self.desired_caps = {
            'platformName': PLATFORM,
            'deviceName': DEVICE_NAME,
            'appPackage': APP_PACKAGE,
            'appActivity': APP_ACTIVITY,
            'noReset': NORESET,
            'newCommandTimeout': TIMEOUT,
        }
        print('打开 appium 服务器...')
        print('配置 appium ...')
        self.driver = webdriver.Remote(DRIVER_SERVER, self.desired_caps)
        self.wait = WebDriverWait(self.driver, 10)
        self.size = self.driver.get_window_size()
        # 设置数据库
        self.client = MongoClient(MONGO_URL)
        self.db = self.client[MONGO_DB]
        self.collection = self.db[MONGO_COLLECTION]


# 屏幕方法
    def swipeUp(self):
        # 向上滑动屏幕
        self.driver.swipe(self.size['width'] * random.uniform(0.55, 0.65),
                          self.size['height'] * random.uniform(0.90, 1),
                          self.size['width'] * random.uniform(0.55, 0.65),
                          self.size['height'] * random.uniform(0.1, 0.25))
        print('向上滑动屏幕')

    def swipeDown(self):
        # 向下滑动屏幕
        self.driver.swipe(self.size['width'] * random.uniform(0.55, 0.65),
                          self.size['height'] * random.uniform(0.25, 0.35),
                          self.size['width'] * random.uniform(0.55, 0.65),
                          self.size['height'] * random.uniform(0.65, 0.75))
        print('向下滑动屏幕')

    def swipeRight(self):
        # 向右滑动屏幕
        self.driver.swipe(self.size['width'] * random.uniform(0.01, 0.11),
                          self.size['height'] * random.uniform(0.75, 0.89),
                          self.size['width'] * random.uniform(0.89, 0.98),
                          self.size['height'] * random.uniform(0.75, 0.89))
        print('向右滑动屏幕')

class Lianjia(Automation):
    
    def __init__(self,APP_PACKAGE, APP_ACTIVITY):
        # 东方财富操作元素
        self.rent_ele = '//*[@text="租房"]'
        self.popup_ele = 'com.matrix.rentplathouse:id/iv_close'
        self.transaction_ele = '//*[@text="查成交价"]'
        self.method_ele = '//*[@text="方式"]'
        #self.wholerent_ele = '//*[@text="不限"]'
        self.wholerent_ele = '/hierarchy/android.widget.FrameLayout/android.widget.LinearLayout/android.widget.FrameLayout/android.widget.FrameLayout/android.widget.LinearLayout/android.widget.LinearLayout/android.support.v4.widget.DrawerLayout/android.widget.RelativeLayout/android.widget.LinearLayout[2]/android.widget.FrameLayout/android.widget.FrameLayout/android.widget.FrameLayout/android.widget.LinearLayout/android.support.v7.widget.RecyclerView[1]/android.widget.LinearLayout[1]/android.widget.TextView'        
        self.view_ele = 'com.matrix.rentplathouse:id/tvFilterSure'     
        self.house_title = 'com.matrix.rentplathouse:id/tvTitle'
        self.house_location = 'com.matrix.rentplathouse:id/tvSubTitle'
        self.house_price = 'com.matrix.rentplathouse:id/tvPrice'
   
        super().__init__(APP_PACKAGE, APP_ACTIVITY)
        
    def login(self): 
        print('打开 {} App ...'.format('链家'))
        print('随机等待几秒钟...')
        time.sleep(random.randint(4,10))
        
    def rent(self):  
        print('点击租房按钮...')
        rent_btn = self.wait.until(EC.presence_of_element_located((By.XPATH,self.rent_ele)))
        rent_btn.click()
#         print('关闭弹窗按钮...')     
#         pop_btn = self.wait.until(EC.presence_of_element_located((By.ID,self.popup_ele)))
#         pop_btn.click()
        time.sleep(random.randint(1,4))
        print('点击查成交价...')
        transaction_btn = self.wait.until(EC.presence_of_element_located((By.XPATH,self.transaction_ele)))
        transaction_btn.click()
        print('方式选择整租不限...')
        time.sleep(random.randint(2,5))
        method_btn = self.wait.until(EC.presence_of_element_located((By.XPATH,self.method_ele)))
        method_btn.click()   
        time.sleep(random.randint(1,3))
        wholerent_btn= self.wait.until(EC.presence_of_element_located((By.XPATH,self.wholerent_ele)))
        wholerent_btn.click()
        print('查看房源...')
        view_btn = self.wait.until(EC.presence_of_element_located((By.ID,self.view_ele)))
        available_info = view_btn.get_attribute('text')
        available_num = int(re.search('\d+',available_info).group())
        print('筛选结果：共有{}套房源'.format(available_num))
        view_btn.click()     
             
    def crawl(self):

        house_list = []
        print('获取房源信息中...')
        page = 0
        while True:
            print('第{}页'.format(page+1))
            time.sleep(random.uniform(4,15))
            house_eles = self.driver.find_elements_by_id('com.matrix.rentplathouse:id/ll_house_info')
            print('当页房源数',len(house_eles)) 
            for ele in house_eles:
                house_dict = dict()
                try:
                    house_dict['title'] = ele.find_element_by_id(self.house_title).get_attribute('text')
                    house_dict['location'] = ele.find_element_by_id(self.house_location).get_attribute('text')
                    house_dict['price'] = int(ele.find_element_by_id(self.house_price).get_attribute('text'))
                    self.collection.update_one({'title':house_dict['title'],'location':house_dict['location']},{'$set':house_dict},True)
                    print('存入数据库中...')
                    print(house_dict)
                except NoSuchElementException:
                    pass           
            self.swipeUp()
            page += 1
#             if page > 4:
#                 break

        
    def main(self):
        self.login()
        self.rent()
        self.crawl()

lianjia_appPackage = 'com.homelink.android'
lianjia_appActivity = 'com.homelink.android.SplashScreenActivity'

if __name__ == '__main__':
    lianjia = Lianjia(lianjia_appPackage,lianjia_appActivity)
    lianjia.main()

打开 appium 服务器...
配置 appium ...
打开 链家 App ...
随机等待几秒钟...
点击租房按钮...
点击查成交价...
方式选择整租不限...
查看房源...
筛选结果：共有97923套房源
获取房源信息中...
第1页
当页房源数 5
存入数据库中...
{'title': '整租·星海名城一期四组团 2室1厅', 'location': '南山区·前海', 'price': 6800}
存入数据库中...
{'title': '整租·卡罗社区 1室1厅', 'location': '宝安区·翻身 距1号线(罗宝线)-宝安中心站1km', 'price': 5000}
存入数据库中...
{'title': '整租·鸿威海怡湾 3室2厅', 'location': '南山区·深圳湾', 'price': 14000}
存入数据库中...
{'title': '整租·鸿荣源·壹城中心第九区 4室2厅', 'location': '龙华区·龙华中心 距4号线(龙华线)-龙华站913m', 'price': 7100}
存入数据库中...
{'title': '整租·翠竹苑 2室1厅', 'location': '罗湖区·百仕达 距3号线(龙岗线)-翠竹站304m', 'price': 4200}
向上滑动屏幕
第2页
当页房源数 6
存入数据库中...
{'title': '整租·国展苑 2室1厅', 'location': '龙岗区·布吉关 距3号线(龙岗线)-布吉站811m', 'price': 3800}
存入数据库中...
{'title': '整租·南油生活B区 1室0厅', 'location': '南山区·后海 距2号线(蛇口线)-登良站937m', 'price': 3000}
存入数据库中...
{'title': '整租·都市翠海华苑 1室0厅', 'location': '宝安区·新安 距5号线(环中线)-灵芝站98m', 'price': 2700}
存入数据库中...
{'title': '整租·皇御苑三期 2室1厅', 'location': '福田区·皇岗', 'price': 7500}
存入数据库中...
{'title': '整租·新一代国际公寓 1室1厅', 'location': '南山区·后海 

存入数据库中...
{'title': '整租·陶然居 2室1厅', 'location': '南山区·华侨城', 'price': 4900}
存入数据库中...
{'title': '整租·青年城邦园 1室1厅', 'location': '龙华区·龙华中心', 'price': 3600}
向上滑动屏幕
第18页
当页房源数 6
存入数据库中...
{'title': '整租·荔馨村南山区 1室1厅', 'location': '南山区·南头', 'price': 4200}
存入数据库中...
{'title': '整租·佳华领汇广场 2室1厅', 'location': '龙岗区·坂田', 'price': 5600}
存入数据库中...
{'title': '整租·玉湖湾 3室1厅', 'location': '宝安区·碧海 距11号线-碧海湾站1.2km', 'price': 6200}
存入数据库中...
{'title': '整租·坪山招商花园城南区 3室2厅', 'location': '坪山区·坪山', 'price': 3700}
存入数据库中...
{'title': '整租·保利悦都 5室2厅', 'location': '龙华区·龙华中心 距4号线(龙华线)-清湖站902m', 'price': 8600}
存入数据库中...
{'title': '整租·卡罗社区 1室1厅', 'location': '宝安区·翻身 距5号线(环中线)-翻身站596m', 'price': 4800}
向上滑动屏幕
第19页
当页房源数 7
存入数据库中...
{'title': '整租·坪山招商花园城南区 3室2厅', 'location': '坪山区·坪山', 'price': 3700}
存入数据库中...
{'title': '整租·保利悦都 5室2厅', 'location': '龙华区·龙华中心 距4号线(龙华线)-清湖站902m', 'price': 8600}
存入数据库中...
{'title': '整租·卡罗社区 1室1厅', 'location': '宝安区·翻身 距5号线(环中线)-翻身站596m', 'price': 4800}
存入数据库中...
{'title': '整租·招商依山郡 1室0厅', 'location': 

向上滑动屏幕
第34页
当页房源数 6
存入数据库中...
{'title': '整租·福盈中央山 4室2厅', 'location': '光明区·公明', 'price': 4000}
存入数据库中...
{'title': '整租·朗泓龙园大观 3室2厅', 'location': '龙岗区·龙岗双龙', 'price': 3000}
存入数据库中...
{'title': '整租·聚龙大厦 2室1厅', 'location': '罗湖区·螺岭', 'price': 4200}
存入数据库中...
{'title': '整租·瑞景华庭 2室1厅', 'location': '南山区·前海 距9号线-荔林站502m', 'price': 5000}
存入数据库中...
{'title': '整租·全海花园 4室2厅', 'location': '福田区·沙尾 距7号线-沙尾站541m', 'price': 10000}
存入数据库中...
{'title': '整租·花样年花郡 1室0厅', 'location': '宝安区·宝安中心', 'price': 3300}
向上滑动屏幕
第35页
当页房源数 7
存入数据库中...
{'title': '整租·中骏四季阳光二期 3室2厅', 'location': '龙岗区·坪地', 'price': 3300}
存入数据库中...
{'title': '整租·鸿隆广场 2室1厅 复式', 'location': '宝安区·西乡', 'price': 3800}
存入数据库中...
{'title': '整租·龙珠花园B区 2室1厅', 'location': '龙岗区·布吉关', 'price': 3800}
存入数据库中...
{'title': '整租·立新湖花园 5室2厅', 'location': '宝安区·福永', 'price': 4500}
存入数据库中...
{'title': '整租·金海岸大厦 1室1厅', 'location': '南山区·后海 距9号线-南油站698m', 'price': 5700}
向上滑动屏幕
第36页
当页房源数 7
存入数据库中...
{'title': '整租·金海岸大厦 1室1厅', 'location': '南山区·后海 距9号线-南油站698m', 'pric

存入数据库中...
{'title': '整租·莲城花园 2室2厅', 'location': '南山区·南头', 'price': 6600}
向上滑动屏幕
第51页
当页房源数 7
存入数据库中...
{'title': '整租·泰阳硅谷公寓 1室0厅', 'location': '南山区·科技园 距1号线(罗宝线)-高新园站825m', 'price': 3800}
存入数据库中...
{'title': '整租·洪浪一村 2室2厅', 'location': '宝安区·新安', 'price': 4300}
存入数据库中...
{'title': '整租·金钟大厦 2室2厅', 'location': '南山区·南山中心', 'price': 7600}
存入数据库中...
{'title': '整租·花样年花乡 2室1厅', 'location': '宝安区·宝安中心 距1号线(罗宝线)-宝体站657m', 'price': 5500}
存入数据库中...
{'title': '整租·中海怡翠山庄三期 1室1厅', 'location': '龙岗区·布吉水径', 'price': 2900}
向上滑动屏幕
第52页
当页房源数 7
存入数据库中...
{'title': '整租·中海怡翠山庄三期 1室1厅', 'location': '龙岗区·布吉水径', 'price': 2900}
存入数据库中...
{'title': '整租·南油A区 1室0厅', 'location': '南山区·后海 距9号线-南油站159m', 'price': 2200}
存入数据库中...
{'title': '整租·家和花园北区 2室2厅', 'location': '龙岗区·坂田', 'price': 3800}
存入数据库中...
{'title': '整租·朗钜御风庭 1室0厅', 'location': '罗湖区·黄贝岭 距9号线-文锦站113m', 'price': 3500}
存入数据库中...
{'title': '整租·财富城 4室2厅', 'location': '坪山区·坪山', 'price': 3000}
向上滑动屏幕
第53页
当页房源数 6
存入数据库中...
{'title': '整租·沙河观景阁 1室0厅', 'location': '南

向上滑动屏幕
第68页
当页房源数 7
存入数据库中...
{'title': '整租·旭飞花园 1室1厅', 'location': '福田区·八卦岭', 'price': 2500}
存入数据库中...
{'title': '整租·诺德假日花园 2室1厅', 'location': '南山区·前海', 'price': 6400}
存入数据库中...
{'title': '整租·金地国际公寓 3室1厅', 'location': '南山区·科技园 距2号线(蛇口线)-科苑站376m', 'price': 9800}
存入数据库中...
{'title': '整租·慢城一期 2室2厅', 'location': '龙岗区·布吉石芽岭', 'price': 4000}
存入数据库中...
{'title': '整租·泰阳硅谷公寓 1室0厅', 'location': '南山区·科技园 距1号线(罗宝线)-高新园站825m', 'price': 3700}
向上滑动屏幕
第69页
当页房源数 7
存入数据库中...
{'title': '整租·君临天下 3室1厅', 'location': '福田区·车公庙 距1号线(罗宝线)-购物公园站895m', 'price': 13000}
存入数据库中...
{'title': '整租·港湾丽都 4室2厅', 'location': '南山区·前海', 'price': 9300}
存入数据库中...
{'title': '整租·信义假日名城六期溢芳园 2室1厅', 'location': '龙岗区·布吉关', 'price': 4500}
存入数据库中...
{'title': '整租·松茂御龙湾 1室1厅', 'location': '宝安区·碧海 距1号线(罗宝线)-西乡站1.1km', 'price': 5900}
存入数据库中...
{'title': '整租·十五峯花园 4室1厅', 'location': '南山区·西丽', 'price': 19000}
向上滑动屏幕
第70页
当页房源数 6
存入数据库中...
{'title': '整租·艺华花园 1室1厅', 'location': '南山区·南头', 'price': 3300}
存入数据库中...
{'title': '整租·百合星城一期 2室1厅'

...

存入数据库中...
{'title': '整租·万科天誉 3室1厅', 'location': '龙岗区·龙岗中心城 距3号线(龙岗线)-吉祥站462m', 'price': 5000}
存入数据库中...
{'title': '整租·名居广场 1室1厅', 'location': '龙岗区·龙岗中心城 距3号线(龙岗线)-爱联站94m', 'price': 3000}
向上滑动屏幕
第501页
当页房源数 7
存入数据库中...
{'title': '整租·TT国际公寓 1室1厅', 'location': '福田区·景田 距2号线(蛇口线)-景田站324m', 'price': 5000}
存入数据库中...
{'title': '整租·恒立听海花园 2室1厅', 'location': '南山区·科技园 距2号线(蛇口线)-科苑站674m', 'price': 8800}
存入数据库中...
{'title': '整租·港丽豪园 3室2厅', 'location': '福田区·福田中心 距1号线(罗宝线)-岗厦站648m', 'price': 12000}
存入数据库中...
{'title': '整租·前海银鸿 2室1厅 复式', 'location': '宝安区·翻身 距5号线(环中线)-翻身站1.2km', 'price': 5100}
存入数据库中...
{'title': '整租·佳兆业水岸新都 2室1厅', 'location': '龙岗区·龙岗双龙', 'price': 3100}
向上滑动屏幕
第502页
当页房源数 7
存入数据库中...
{'title': '整租·方鼎华庭 3室1厅', 'location': '南山区·南头', 'price': 7000}
存入数据库中...
{'title': '整租·雅景居 2室1厅', 'location': '宝安区·新安', 'price': 5300}
存入数据库中...
{'title': '诺德中央道 3室1厅', 'location': '龙岗区·坂田 距5号线(环中线)-坂田站1.1km', 'price': 5500}
存入数据库中...
{'title': '整租·金城华庭 1室1厅', 'location': '罗湖区·螺岭 距2号线(蛇口线)-黄贝岭站742m', 'pric

向上滑动屏幕
第519页
当页房源数 6
存入数据库中...
{'title': '整租·宇宏健康花城 3室1厅', 'location': '龙岗区·布吉石芽岭', 'price': 5300}
存入数据库中...
{'title': '整租·翠枫豪园 3室1厅', 'location': '龙岗区·布吉街', 'price': 2000}
存入数据库中...
{'title': '整租·阳光棕榈园三期 3室1厅', 'location': '南山区·前海 距1号线(罗宝线)-大新站921m', 'price': 9500}
存入数据库中...
{'title': '整租·鹏莲花园 2室1厅', 'location': '罗湖区·莲塘', 'price': 3600}
存入数据库中...
{'title': '整租·新地中央 3室2厅', 'location': '光明区·公明', 'price': 5500}
向上滑动屏幕
第520页
当页房源数 6
存入数据库中...
{'title': '整租·宇宏健康花城 3室1厅', 'location': '龙岗区·布吉石芽岭', 'price': 5300}
存入数据库中...
{'title': '整租·翠枫豪园 3室1厅', 'location': '龙岗区·布吉街', 'price': 2000}
存入数据库中...
{'title': '整租·阳光棕榈园三期 3室1厅', 'location': '南山区·前海 距1号线(罗宝线)-大新站921m', 'price': 9500}
存入数据库中...
{'title': '整租·鹏莲花园 2室1厅', 'location': '罗湖区·莲塘', 'price': 3600}
存入数据库中...
{'title': '整租·新地中央 3室2厅', 'location': '光明区·公明', 'price': 5500}
向上滑动屏幕
第521页
当页房源数 6
存入数据库中...
{'title': '整租·宇宏健康花城 3室1厅', 'location': '龙岗区·布吉石芽岭', 'price': 5300}
存入数据库中...
{'title': '整租·翠枫豪园 3室1厅', 'location': '龙岗区·布吉街', 'price': 2000}
存

向上滑动屏幕
第539页
当页房源数 6
存入数据库中...
{'title': '整租·宇宏健康花城 3室1厅', 'location': '龙岗区·布吉石芽岭', 'price': 5300}
存入数据库中...
{'title': '整租·翠枫豪园 3室1厅', 'location': '龙岗区·布吉街', 'price': 2000}
存入数据库中...
{'title': '整租·阳光棕榈园三期 3室1厅', 'location': '南山区·前海 距1号线(罗宝线)-大新站921m', 'price': 9500}
存入数据库中...
{'title': '整租·鹏莲花园 2室1厅', 'location': '罗湖区·莲塘', 'price': 3600}
存入数据库中...
{'title': '整租·新地中央 3室2厅', 'location': '光明区·公明', 'price': 5500}
向上滑动屏幕
第540页
当页房源数 6
存入数据库中...
{'title': '整租·宇宏健康花城 3室1厅', 'location': '龙岗区·布吉石芽岭', 'price': 5300}
存入数据库中...
{'title': '整租·翠枫豪园 3室1厅', 'location': '龙岗区·布吉街', 'price': 2000}
存入数据库中...
{'title': '整租·阳光棕榈园三期 3室1厅', 'location': '南山区·前海 距1号线(罗宝线)-大新站921m', 'price': 9500}
存入数据库中...
{'title': '整租·鹏莲花园 2室1厅', 'location': '罗湖区·莲塘', 'price': 3600}
存入数据库中...
{'title': '整租·新地中央 3室2厅', 'location': '光明区·公明', 'price': 5500}
向上滑动屏幕
第541页
当页房源数 6
存入数据库中...
{'title': '整租·宇宏健康花城 3室1厅', 'location': '龙岗区·布吉石芽岭', 'price': 5300}
存入数据库中...
{'title': '整租·翠枫豪园 3室1厅', 'location': '龙岗区·布吉街', 'price': 2000}
存

向上滑动屏幕
第559页
当页房源数 6
存入数据库中...
{'title': '整租·宇宏健康花城 3室1厅', 'location': '龙岗区·布吉石芽岭', 'price': 5300}
存入数据库中...
{'title': '整租·翠枫豪园 3室1厅', 'location': '龙岗区·布吉街', 'price': 2000}
存入数据库中...
{'title': '整租·阳光棕榈园三期 3室1厅', 'location': '南山区·前海 距1号线(罗宝线)-大新站921m', 'price': 9500}
存入数据库中...
{'title': '整租·鹏莲花园 2室1厅', 'location': '罗湖区·莲塘', 'price': 3600}
存入数据库中...
{'title': '整租·新地中央 3室2厅', 'location': '光明区·公明', 'price': 5500}
向上滑动屏幕
第560页
当页房源数 6
存入数据库中...
{'title': '整租·宇宏健康花城 3室1厅', 'location': '龙岗区·布吉石芽岭', 'price': 5300}
存入数据库中...
{'title': '整租·翠枫豪园 3室1厅', 'location': '龙岗区·布吉街', 'price': 2000}
存入数据库中...
{'title': '整租·阳光棕榈园三期 3室1厅', 'location': '南山区·前海 距1号线(罗宝线)-大新站921m', 'price': 9500}
存入数据库中...
{'title': '整租·鹏莲花园 2室1厅', 'location': '罗湖区·莲塘', 'price': 3600}
存入数据库中...
{'title': '整租·新地中央 3室2厅', 'location': '光明区·公明', 'price': 5500}
向上滑动屏幕
第561页
当页房源数 6
存入数据库中...
{'title': '整租·宇宏健康花城 3室1厅', 'location': '龙岗区·布吉石芽岭', 'price': 5300}
存入数据库中...
{'title': '整租·翠枫豪园 3室1厅', 'location': '龙岗区·布吉街', 'price': 2000}
存

InvalidElementStateException: Message: Swipe did not complete successfully


为了去除重复，这里调用 update() 方法

`self.collection.update({'nickname':nickname,'content':content},{'$set':data},True)`

根据昵称和正文来查询信息，如果信息不存在，则插入数据，否则更新数据。这个操作的关键点在第三个参数 True，此参数设置为 True，这可以实现存在即更新、不存在则插入操作。

In [21]:
random.uniform(2,10)

5.288072275463582